# **Tutorial: AdaBoost**
### By Kostas Hatalis

**Prerequisite Notebooks:** *Decision Trees, Ensemble Learning*

___
## **Boosting**

The concept of boosting, introduced in 1990 by Robert Schapire [1], is an ensemble approach in which several models are trained sequentially with each model learning from the errors of its predecessors. In bagging we wanted low bias, high variance models. In boosting we want high bias models, ie **weak learners** with low accuracy 50-60% (i.e., models that are only slightly better than random guessing). Boosting then aims to reduce bias, and to a lesser degree reduce variance; it converts weak learners to strong ones. Hypothesis is that combining many weak learners can yield a stronger learning. 

An example of a weak learner is a CART with a max depth of 1, also called **decision stumps**. This only splits the space in half. Another weak learner is logistic regression. The main idea is to train an ensemble of weak learners sequentially. Then each predictor tries to correct its predecessor. The predictions from all of them are then combined through a weighted majority vote (or sum) to produce the final prediction.

## **Adaboost**

The popular boosting algorithm AdaBoost, short for Adaptive Boosting, was introduced in 1995 by Freund and Schapire [2] and adopts to weak learners. AdaBoost was the first really successful boosting algorithm developed for binary classification. Modern boosting methods build on AdaBoost, most notably stochastic gradient boosting machines.

The core principle of AdaBoost is to fit a sequence of weak learners on repeatedly modified versions of the data. The predictions from all of them are then combined through a weighted majority vote (or sum) to produce the final prediction. The data modifications at each so-called boosting iteration consist of applying weights $w_1, w_2, ..., w_N$ to each of the training samples. Each predictor pays more attention to the instances wrongly predicted by its predecessor. 

**Advantages:**
1. Less prone to overfitting.
2. Requires less hyperparameter tuning compared to more advanced methods like SVMs or ANNs.

**Disadvantages:**
1. Sensitive to noisy data 
2. Sensitive to outliers.

### **Algorithm & Mathematics**

Main idea:

0. With AdaBoost we typically use targets $y_i =$ {-1,+1}, and not {0,1}. We can convert logistic targets to -1 and +1 with the equation $ 2 \cdot y_i -1 $.
1. Assign every observation, $(x_i,y_i)$, an initial weight value $w_i = 1/N$, where $N$ is the total number of observations.
2. Train a base weak model $m=1,...,M$ (typically a decision stump), on all data (no resampling/bootstrapping). After training assign a weight $\alpha_m$ to the model based on accuracy. A more accurate model is assigned higher weight so that it will have more impact in final outcome.
3. For each $(x_i,y_i)$ observation
    - If predicted correctly, $w_i$ is increased.
    - If predicted incorrectly, $w_i$ is decreased.
4. Train a new base weak model (called additive modeling) where observations with greater weights are given more priority.
5. Repeat steps 3 and 4 until observations are perfectly predicted or a preset number of weak learners are trained.
6. Now you are left with a pool of trained weak learners, each with a stage value.
    - For classification, the final label is obtained from the ensemble's predictions by weighted majority voting. For regression we use weighted average.

For classification, the final prediction is made as a weighted sum of all the trained weak learners:

$$
F(x_i) = sign \left( \sum_{m=1}^{M} \alpha_m f_{m}(x_i) \right)
$$

where $f_{m}(x)$ is the output of weak classifier $m$ for input $x_i$ and $\alpha_m$ is the weight assigned to weak classifier $m$. We calculate $\alpha_m$ on the predictor's training error rate $\epsilon_m$ as follows:

$$
\alpha_m = \frac{1}{2} \ln \frac{1-\epsilon_m}{\epsilon_m}
$$
$$
\alpha_m =  \eta \cdot \alpha_m
$$

where $0<\eta \leq 1$ is the learning rate that is used to shrink the coefficient $\alpha_m$ of a trained predictor. It's important to note that there's a tradeoff between $\eta$ and the number of estimators. A smaller value of $\eta$ should be compensated by a greater number of estimators. Note that we calculate the error (misclassification) rate like so:

$$
\epsilon_m = \frac{1}{N} \sum_{i=1}^{N} I(y_i \neq \hat{y}_i)
$$

For regression we use MSE for the error rate. Here's a plot of what $alpha_tm$ will look like for classifiers with different error rates (from [3]):

<img src="images/curve.PNG" width="300">

After weak classifier is trained, we update the weight of each training example with following formula

$$
W_{i}^{(m+1)} = \frac{ W_{i}^{(M)} e^{ -\alpha_m y_i f_m (x_i) } }{\sum_{i=1}^{N} w_{i}^{(M)}}
$$

where $W_{i}^{(m)}$ is a vector of weights, with one weight for each training example in the training set. This equation shows you how to update the weight for the ith training example. The process of AdaBoost is illustrated below (from [4]):

<img src="images/adaboost_eta.PNG" width="700">

$\alpha$ is used to determine the training weights $w_i$ of the next predictor. Notice how the incorrectly predicted instances shown in green acquir higher weights. The next predictor is then forced to pay more attention to the incorrectly predicted instances. This process is reapeted sequentially, until $M$ predictors forming the ensemble are trained.


### **TUTORIAL ADABOOST**

Sklearn implements AdaBoost this with the functions `AdaBoostClassifier` and `AdaBoostRegressor`. The main hyperparameters to tune are:

`base_estimator` is the learning algorithm to use to train the weak models. The default argument is a decision stump, and it's typically the best choice.

`n_estimators` is the number of models to iteratively train. Default is 50. I recommend doing a CV grid search 100 to 400.

`learning_rate` is the contribution of each model to the weights and defaults to 1. Reducing the learning rate will mean the weights will be increased or decreased to a small degree, forcing the model train slower (but sometimes resulting in better performance scores). You should start with a low learning rate maybe 0.001, 0.01, or 0.1. This will make your model more robust and hence you will be able to control the variance in your dev/test sets. 

In [1]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets  import load_breast_cancer
from sklearn.model_selection import cross_val_score

# load data
data = load_breast_cancer()
X, y = data.data, data.target

# Instantiate a classification-tree 'dt'
dt = DecisionTreeClassifier(random_state=1)

# Instantiate a AdaBoost Classifier 'ac'
ac = AdaBoostClassifier(random_state=1)

# Get accuracy score using CV for dt
scores = cross_val_score(dt, X, y, cv=5, scoring='accuracy')
print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'Decision Tree'))

# Get accuracy score using CV for ac
scores = cross_val_score(ac, X, y, cv=5, scoring='accuracy')
print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'AdaBoost'))

Accuracy: 0.92 (+/- 0.02) [Decision Tree]
Accuracy: 0.97 (+/- 0.02) [AdaBoost]


___
## **Which is the best:** Bagging or Boosting?

There’s not an outright winner; it depends on the data, the simulation and the circumstances.
Bagging and Boosting decrease the variance of your single estimate as they combine several estimates from different models. So the result may be a model with higher stability.

If the problem is that the single model gets a very low performance, Bagging will rarely get a better bias. However, Boosting could generate a combined model with lower errors as it optimises the advantages and reduces pitfalls of the single model.

By contrast, if the difficulty of the single model is over-fitting, then Bagging is the best option. Boosting for its part doesn’t help to avoid over-fitting; in fact, this technique is faced with this problem itself. For this reason, Bagging is effective more often than Boosting.


___
## **References**

[1] Schapire, Robert E. (1990). "The Strength of Weak Learnability". Machine Learning. 5 (2): 197–227.

[2] Freund, Yoav, and Robert E. Schapire. "A decision-theoretic generalization of on-line learning and an application to boosting." Journal of computer and system sciences 55.1 (1997): 119-139.
    
[3] http://mccormickml.com/2013/12/13/adaboost-tutorial/

[4] https://campus.datacamp.com/courses/machine-learning-with-tree-based-models-in-python/  

[5] https://en.wikipedia.org/wiki/Boosting_(machine_learning)

[6] https://scikit-learn.org/stable/modules/ensemble.html#adaboost

[7] https://machinelearningmastery.com/boosting-and-adaboost-for-machine-learning/

[8] http://mccormickml.com/2013/12/13/adaboost-tutorial/